In [2]:
import numpy as np

# network size N
N = 5

# number of edges
M = N*3

## initialize time
T = 0
## choose Tmax
Tmax = 5

# network size N
N = 5

# number of edges
M = N*3

Tmax = 5


In [24]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

## inputs are the initial condition and the edge matrix
def find_tranformation_matrix(b, E):

    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    X = np.identity(N, dtype=bool)
    
    vf = np.vectorize(lambda x: 2**x)
    ints = np.array(list(range(N)))
    I = vf(ints)

    counts = Counter()
    for element in I:
        counts.increment(element, 1)

    ## initialize the number of distinct rows to N (start with Identity matrix)
    distinct_rows = N

    # run simulation
    for t in range(n):
        
        ## recovery event occurs (turns diagonal entry to 0 and premultiplies A) == row i of A is set to False 
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            print(f"recovery event at iteration {t} of node {i}")
        
            row_i_not_distinct = I[i] in counts and counts[I[i]] > 1

            ## if the row is already all zeros, nothing needs to be done (no change in distinct rows)
            if I[i] != 0:

                if 0 in counts and counts[0] >= 1:
                    ## if there is at least one non-zero row, and the row being zeroed was distinct, then the number of distinct rows decreases by 1
                    if row_i_not_distinct == False:

                        distinct_rows -= 1
                    ## if there is at least one non zero row and the row being zeroed was not distinct, then the number of distinct rows remains the same
                     
                else:
                    ## if there are no nonzero rows, and the row being zeroed was not distinct, then the number of distinct rows increases by 1
                    if row_i_not_distinct == True:
                            
                        distinct_rows = min(distinct_rows + 1, N)
                    ## if there are no nonzero rows, and the row being zeroed was distinct, then the number of distinct rows remains the same
                
                counts.increment(I[i], -1)

                X[i, :] = False

                I[i] = 0

                counts.increment(0, 1)
        else:

            i,j = E[np.random.randint(M)]

            print(f"spreading event at iteration {t} of nodes {i} and {j}")

            if I[i] != 0 or I[j] != 0:

                if I[i] != I[j]:
                    
                    new = int(''.join('1' if x else '0' for x in np.logical_or(X[i], X[j]))[::-1], 2)
                    counts.increment(I[i], -1)
                    counts.increment(I[j], -1)
                    new_not_distinct = new in counts and counts[new] >= 1
                    print(new_not_distinct)
                    
                    row_i_not_distinct = I[i] in counts and counts[I[i]] > 1
                    print(row_i_not_distinct)

                    row_j_not_distinct = I[j] in counts and counts[I[j]] > 1
                    print(row_j_not_distinct)

                    counts.increment(new, 1)
                    I[i] = I[j] = new

                    if new_not_distinct == False:
                        if row_i_not_distinct == False and row_j_not_distinct == False:
                            distinct_rows -= 1
                        if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                            distinct_rows = distinct_rows
                        if row_i_not_distinct == True and row_j_not_distinct == True:
                            distinct_rows = min(distinct_rows + 1, N)

                    else:
                        if row_i_not_distinct == False and row_j_not_distinct == False:
                            distinct_rows -= 2
                        if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                            distinct_rows -= 1


                X[i] = X[j] = np.logical_or(X[i], X[j])

        print(distinct_rows)
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2)
        if distinct_rows == 2:
            
            break

    return X, I


In [25]:
b = 0.5

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

x0 = np.ones(N, dtype=bool)
matrix_A = find_tranformation_matrix(b, E)
print(matrix_A)


spreading event at iteration 0 of nodes 2 and 3
False
False
False
4
spreading event at iteration 1 of nodes 4 and 1
False
False
False
3
recovery event at iteration 2 of node 0
3
spreading event at iteration 3 of nodes 4 and 1
3
spreading event at iteration 4 of nodes 0 and 2
False
False
False
2
(array([[False, False,  True,  True, False],
       [False,  True, False, False,  True],
       [False, False,  True,  True, False],
       [False, False,  True,  True, False],
       [False,  True, False, False,  True]]), array([12, 18, 12, 12, 18]))


In [27]:
X = np.array([[False, False,  True,  True, False],
       [False,  True, False, False,  True],
       [False, False,  True,  True, False],
       [False, False,  True,  True, False],
       [False,  True, False, False,  True]])

print(int(''.join('1' if x else '0' for x in X[0])[::-1], 2))
print(int(''.join('1' if x else '0' for x in X[3])[::-1], 2))


12
12


In [23]:
## initialize boolean identity Matrix A of dimensions NxN
X = np.identity(N, dtype=bool)
    
vf = np.vectorize(lambda x: 2**x)
ints = np.array(list(range(N)))
I = vf(ints)

counts = Counter()
for element in I:
    counts.increment(element, 1)

print(X)
print(I)
print(counts)


# new = int(''.join('1' if x else '0' for x in np.logical_or(X[i], X[j])), 2)

print(int('10000'[::-1], 2))


[[ True False False False False]
 [False  True False False False]
 [False False  True False False]
 [False False False  True False]
 [False False False False  True]]
[ 1  2  4  8 16]
{1: 1, 2: 1, 4: 1, 8: 1, 16: 1}
1


In [8]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val


vf = np.vectorize(lambda x: 2**x)
ints = np.array(list(range(N)))
I = vf(ints)

counts = Counter()
for element in I:
    counts.increment(element, 1)

counts.increment(1, 1)
print(len(counts))
print(counts)
print(counts[1])

row_i_not_distinct = I[1] in counts and counts[I[1]] > 1
print(row_i_not_distinct)

0 in counts and counts[0] >= 1


5
{2: 1, 4: 1, 8: 1, 16: 1, 1: 2}
2
False


False